# Structure
1. classic pipeline, params hardcoded everywhere
2. parametrizable function, introduce DictConfig
3. hydra decorates, introduce configuration file
4. introduce override syntax commandline
5. introduce sweeps (e.g. multiple seeds), sequential
6. introduce launchers (local: joblib, but also slurm/submitit)
7. introduce composition
8. introduce instantiation

# AutoML Fall School 2023 Hydra Hands-On

Welcome to our tutorial session on [hydra](hydra.cc)! 🐍
Hydra is a tool for configuring and running your experiments and optimization is seemlessly integrated.

Hydra can:

* Hierarchical configuration composable from multiple sources
* Configuration can be specified or overridden from the command line
* Dynamic command line tab completion
* Run your application locally or launch it to run remotely
* Run multiple jobs with different arguments with a single command



## A Classical Training Pipeline

(This part of the tutorial is the same as in the SMAC tutorial).

We'll start with your classical optimization task.
The task is to optimize the hyperparameters of a [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) on the [digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) dataset. Usually we have some training pipeline with a dataset, a configured model[^1] and some validation procedure to check for generalization performance like this:


[^1]: If we check out the documentation, we will see that loads of design decisions (hyperparameters) are already set to a default value for us.

In [1]:
from IPython.display import Code
Code(filename="hydra_tutorial/classic_pipeline_hardcoded.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

def train_mlp() -> float:
    # we want to have reproducible training
    np.random.seed(seed=1234)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, activation='relu', solver='adam')
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    return np.mean(scores) # mean accuracy over folds

if __name__ == "__main__":
    # Ignore the warnings for now:)
    cv_loss = train_mlp()
    print(f'Cross_validation accuaracy on digits {cv_loss}')

In [7]:
import subprocess
subprocess.run("python hydra_tutorial/classic_pipeline_hardcoded.py".split(" "))

/home/numina/Documents/repos/hydra_tutorial


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

Cross_validation accuaracy on digits 0.9625795297372062


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline_hardcoded.py'], returncode=0)


You can ignore the errors above regarding not converging for now.

What we can see in this example is that we hardcoded many (hyper-)parameters. But maybe we would like to vary them? So let's adapt our `train_mlp` function!
We will use a dict-like object to hold all our parameters.

In [8]:
Code(filename="hydra_tutorial/classic_pipeline_stillhardcoded.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from omegaconf import DictConfig
from rich import print as printr
from rich import inspect

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


def train_mlp(cfg: DictConfig) -> float:
    # we want to have reproducible training
    np.random.seed(seed=cfg.seed)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(hidden_layer_sizes=cfg.hidden_layer_sizes, max_iter=cfg.max_iter, activation=cfg.activation, solver=cfg.solver)
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    return np.mean(scores) # mean accuracy over folds

if __name__ == "__main__":
    # Ignore the warnings for now:)
    # We can easily create a DictConfig object with dict-like syntax
    # We can have almost any type in here
    cfg = DictConfig({
        "seed": 1234,
        "hidden_layer_sizes": (100,),
        "max_iter": 100,
        "activation": 'relu',
        "solver": 'adam',
    })
    inspect(cfg)
    
    cv_loss = train_mlp(cfg=cfg)
    print(f'Cross_validation accuaracy on digits {cv_loss}')

In [9]:
subprocess.run("python hydra_tutorial/classic_pipeline_stillhardcoded.py".split(" "))

╭───────────────── <class 'omegaconf.dictconfig.DictConfig'> ──────────────────╮
│ ╭──────────────────────────────────────────────────────────────────────────╮ │
│ │ {'seed': 1234, 'hidden_layer_sizes': [100], 'max_iter': 100,             │ │
│ │ 'activation': 'relu', 'solver': 'adam'}                                  │ │
│ ╰─────────────────────────��────────────────────────────────────────────────╯ │
│                                                                              │
│         activation = 'relu'                                                  │
│ hidden_layer_sizes = [100]                                                   │
│           max_iter = 100                                                     │
│               seed = 1234                                                    │
│             solver = 'adam'                                                  │
╰────────────────────────────��─────────────────────────────────────────────────╯


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

Cross_validation accuaracy on digits 0.9625795297372062


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline_stillhardcoded.py'], returncode=0)

That's nice but let's vary the parameters with hydra!
Hydra can wrap your main function and pass parameters from the command line or configuration files for you -- without the hassle of writing an argument parser.

In [12]:
Code(filename="hydra_tutorial/classic_pipeline.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from omegaconf import DictConfig
import hydra

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


@hydra.main(version_base=None, config_path="configs", config_name="base")
def train_mlp(cfg: DictConfig) -> float:
    # we want to have reproducible training
    np.random.seed(seed=cfg.seed)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(hidden_layer_sizes=cfg.hidden_layer_sizes, max_iter=cfg.max_iter, activation=cfg.activation, solver=cfg.solver)
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    mean_score = np.mean(scores)

    print(f"Mean accuracy: {mean_score:.4f}")

    return mean_score # mean accuracy over folds

if __name__ == "__main__":
    train_mlp()

Before we let it run, let's have a look at the configuration file.
This will be our default as decorated by hydra.

In [14]:
Code(filename="hydra_tutorial/configs/base.yaml")

# @package _global_
defaults:
  - _self_
  - override hydra/launcher: joblib
  - override hydra/job_logging: colorlog
  - override hydra/hydra_logging: colorlog

seed: 1234
hidden_layer_sizes: [100,]
max_iter: 100
activation: 'relu'
solver: 'adam'

In [13]:
subprocess.run("python hydra_tutorial/classic_pipeline.py".split(" "))

/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

Mean accuracy: 0.9626


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py'], returncode=0)

Now we want to pass arguments via the commandline.
This is easily possible with the [override syntax](https://hydra.cc/docs/advanced/override_grammar/basic/).
Let's vary the hidden layer sizes.

In [19]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[10,10,10]".split(" "))

/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

Mean accuracy: 0.8945


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[10,10,10]'], returncode=0)

But what really comes in handy is the ability to grid of parameter settings.
First, to factor out randomness we want to run different seeds.
Then, we would like to check different settings, e.g. different hidden layer sizes and different activation functions.

For this we will add a list of parameter values and the flag `-m` or `--multirun` to indicate that this is a grid.

In [24]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[100],[10,10,10] -m".split(" "))

[2023-11-02 22:23:42,063][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 2 jobs
[2023-11-02 22:23:42,063][HYDRA] Launching jobs, sweep output dir : multirun/2023-11-02/22-23-41
[2023-11-02 22:23:42,063][HYDRA] 	#0 : hidden_layer_sizes=[100]
[2023-11-02 22:23:42,063][HYDRA] 	#1 : hidden_layer_sizes=[10,10,10]


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimiza

Mean accuracy: 0.8945


/home/numina/micromamba/envs/hydratutorial/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Mean accuracy: 0.9626


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[100],[10,10,10]', '-m'], returncode=0)